In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_excel('./data/ViratKohli-ODI.xlsx')

In [3]:
df.head()

,Match,Innings,Date,M/Inns,Posn,Versus,Ground,How Dismissed,Runs,B/F,S/R,Progressive-Runs,Progressive-B/F,Progressive-Avg,Progressive-S/R
0,1,1.0,2008-08-18,1st,2,Sri Lanka,Rangiri Dambulla International Stadium,lbw b K M D N Kulasekara,12,22,54.55,12,22,12.00,54.55
1,2,2.0,2008-08-20,2nd,2,Sri Lanka,Rangiri Dambulla International Stadium,c C K Kapugedera b T Thushara,37,67,55.22,49,89,24.50,55.06
2,3,3.0,2008-08-24,1st,1,Sri Lanka,R Premadasa Stadium,run out,25,38,65.79,74,127,24.67,58.27
3,4,4.0,2008-08-27,1st,1,Sri Lanka,R Premadasa Stadium,b T Thushara,54,66,81.82,128,193,32.00,66.32
4,5,5.0,2008-08-29,2nd,1,Sri Lanka,R Premadasa Stadium,lbw b K M D N Kulasekara,31,46,67.39,159,239,31.80,66.53


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Match             308 non-null    object        
 1   Innings           296 non-null    float64       
 2   Date              308 non-null    datetime64[ns]
 3   M/Inns            304 non-null    object        
 4   Posn              308 non-null    object        
 5   Versus            308 non-null    object        
 6   Ground            308 non-null    object        
 7   How Dismissed     308 non-null    object        
 8   Runs              308 non-null    object        
 9   B/F               308 non-null    object        
 10  S/R               308 non-null    object        
 11  Progressive-Runs  308 non-null    int64         
 12  Progressive-B/F   308 non-null    int64         
 13  Progressive-Avg   308 non-null    float64       
 14  Progressive-S/R   308 non-

In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Innings,296.0,148.500000,85.592056,1.00,74.7500,148.50,222.2500,296.00
Progressive-Runs,308.0,6910.480519,4375.897862,12.00,2997.2500,6473.00,11105.0000,14557.00
Progressive-B/F,308.0,7578.116883,4596.356583,22.00,3581.7500,7195.00,11923.5000,15543.00
Progressive-Avg,308.0,52.078214,6.768001,12.00,49.1650,52.17,57.8175,60.31
Progressive-S/R,308.0,87.948377,6.428469,54.55,83.9075,89.91,92.9300,93.85


### cleaning "Runs"

In [6]:
runs = []
for i in df['Runs']:
    if str(i).rfind('*')!= -1:
        runs.append(i.split('*')[0])
    elif str(i).rfind('-')!= -1:
        runs.append(0)
    elif str(i).rfind('*') == -1:
        runs.append(i)
df['Runs'] = runs
df['Runs'] = df['Runs'].astype('int64')

### Adding "Captaincy"

In [7]:
captain = []
for i in df['Match']:
    if i.rfind('(c)')!=-1:
        captain.append('Yes')
    elif i.rfind('(c)')==-1:
        captain.append('No')
df['Captain'] = captain

### Cleaning "B/F"

In [8]:
df['B/F'].dtype

dtype('O')

In [9]:
bf = []
for i in df['B/F']:
    if str(i) == '-':
        bf.append(0)
    else:
        bf.append(i)
df['B/F'] = bf
df['B/F'] = df['B/F'].astype('int64')

### Cleaning "S/R"

In [10]:
df['S/R'].dtype

dtype('O')

In [11]:
sr = []
for i in df['S/R']:
    if str(i) == '-':
        sr.append(0)
    else:
        sr.append(i)
df['S/R'] = sr
df['S/R'] = df['S/R'].astype('float64')

### Creating "Dismissal Type"

In [12]:
dt = []
for i in df['How Dismissed']:
    if (i == 'run out'):
        dt.append(i) 
    elif str(i).rfind('hit wicket') != -1:
        dt.append('hit wicket')
    elif (i != 'did not bat') and (i!='not out'):
        dt.append(i.split(' ')[0])
    else:
        dt.append(i)
        
df['Dismissal Type'] = dt
df['Dismissal Type'] = df['Dismissal Type'].map({'lbw':'LBW' , 'c':'Caught',
                                                'run out':'Run Out' , 'b':'Bowled',
                                                'not out':'Not Out', 'did not bat':'Did Not Bat',
                                                'st':'Stumped', 'hit wicket':'Hit Wicket'})

### Cleaning "Innings"

In [13]:
df['M/Inns'].fillna('N/A - No Result',inplace=True,axis=0)

### Cleaning "Posn"

In [14]:
df.Posn.unique()

array([2, 1, 7, 4, '-', 3, 5, 6], dtype=object)

In [15]:
df['Posn'] = df['Posn'].replace({'-':0})

### Nulls

In [16]:
df.isnull().sum()

Match                0
Innings             12
Date                 0
M/Inns               0
Posn                 0
Versus               0
Ground               0
How Dismissed        0
Runs                 0
B/F                  0
S/R                  0
Progressive-Runs     0
Progressive-B/F      0
Progressive-Avg      0
Progressive-S/R      0
Captain              0
Dismissal Type       0
dtype: int64

### Nulls in innings indicate that we have a result for the match but Virat Kohli did not bat. 

In [17]:
df.head()

,Match,Innings,Date,M/Inns,Posn,Versus,Ground,How Dismissed,Runs,B/F,S/R,Progressive-Runs,Progressive-B/F,Progressive-Avg,Progressive-S/R,Captain,Dismissal Type
0,1,1.0,2008-08-18,1st,2,Sri Lanka,Rangiri Dambulla International Stadium,lbw b K M D N Kulasekara,12,22,54.55,12,22,12.00,54.55,No,LBW
1,2,2.0,2008-08-20,2nd,2,Sri Lanka,Rangiri Dambulla International Stadium,c C K Kapugedera b T Thushara,37,67,55.22,49,89,24.50,55.06,No,Caught
2,3,3.0,2008-08-24,1st,1,Sri Lanka,R Premadasa Stadium,run out,25,38,65.79,74,127,24.67,58.27,No,Run Out
3,4,4.0,2008-08-27,1st,1,Sri Lanka,R Premadasa Stadium,b T Thushara,54,66,81.82,128,193,32.00,66.32,No,Bowled
4,5,5.0,2008-08-29,2nd,1,Sri Lanka,R Premadasa Stadium,lbw b K M D N Kulasekara,31,46,67.39,159,239,31.80,66.53,No,LBW


### Save Clean data

In [18]:
# df.to_excel('./Data/ODI-Clean.xlsx',index=False)